In [ ]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

# --- 1. Cargar bandas VV y VH desde archivos tif ---
vv_path = 'VV.tif'
vh_path = 'VH.tif'

with rasterio.open(vv_path) as vv_src, rasterio.open(vh_path) as vh_src:
    vv = vv_src.read(1)
    vh = vh_src.read(1)
    profile = vv_src.profile

# Apilar bandas para crear imagen multibanda
img = np.stack([vv, vh], axis=-1)  # (H, W, 2)

# --- 2. Definir manualmente datos de entrenamiento (fila, columna, clase) ---
# Ejemplo: (fila, columna, clase) — clases: 1=Mombasa, 2=Suelo, 3=Árbol
training_samples = [
    (50, 60, 1), (52, 62, 1),  # Mombasa
    (100, 120, 2), (102, 118, 2),  # Suelo
    (30, 40, 3), (32, 42, 3)  # Árbol
]

X_train = []
y_train = []

for row, col, label in training_samples:
    spectrum = img[row, col]
    X_train.append(spectrum)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# --- 3. Entrenar clasificador Random Forest ---
clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)

# --- 4. Clasificar toda la imagen ---
H, W, _ = img.shape
pixels = img.reshape(-1, 2)

# Enmascarar píxeles inválidos (e.g., 0)
valid_mask = np.all(pixels > 0, axis=1)
classified = np.zeros(pixels.shape[0], dtype=np.uint8)

valid_pixels = pixels[valid_mask]
classified_pixels = clf.predict(valid_pixels)
classified[valid_mask] = classified_pixels

# Reconstruir imagen clasificada
classified_image = classified.reshape(H, W)

# --- 5. Guardar imagen clasificada ---
profile.update(dtype='uint8', count=1)
with rasterio.open('clasificacion_local.tif', 'w', **profile) as dst:
    dst.write(classified_image, 1)

# --- 6. Visualizar resultados ---
plt.figure(figsize=(10, 6))
plt.imshow(classified_image, cmap='Set1')
plt.title("Clasificación supervisada (VV + VH)")
plt.colorbar(label='Clase')
plt.axis('off')
plt.show()
